# NON ROTATING CASE IN ENTANGLED RELATIVITY FOR MAGNETICALLY CHARGED BLACK HOLES

For the details of the field equations in the theory of entangled relativity, we invite 
the reader to refer to O. Minazzoli and E. Santos (2021) - The European Physical Journal describing the case of an electrically charged non rotatin black hole. 

In the article C.F.E. Holzhey & F. Wilczek (1992)- Nuclear Physics B, the authors deduce the magnetically charged solutions from the electric ones by making the following changes:

$\Psi\rightarrow\Psi'=\frac{1}{\Psi}=\left[e^{2\alpha\phi}\right]_{|_{\alpha=(2\sqrt{3})^{-1}}}=\left[\left(1-\frac{r_{-}}{r}\right)^{\frac{-2\alpha^{2}}{1+\alpha^{2}}}\right]_{|_{\alpha=(2\sqrt{3})^{-1}}}=\left(1-\frac{r_{-}}{r}\right)^{\frac{2}{13}}$

and

$F_{\lambda\rho}^{el}\rightarrow F^{mag}_{\mu\nu}=e^{-2\alpha\phi}\frac{1}{2}\epsilon^{\lambda\rho}_{\mu\nu}F_{\lambda\rho}^{el}$ where $F_{\lambda\rho}^{el}$ is the electrically charged solution

The complexity of some calculations in this notebook prompted us to perform intermediate first order Taylor expansions to maintain tractability of the different operations using the Sagemath language. This does not affect the validity of our results since they are part of a first-order analysis in 'a'.

In [1]:
version()

'SageMath version 10.1, Release Date: 2023-08-20'

In [2]:
%display latex

'SageMath version used is 10.1, Release Date: 2023-08-20'

In [3]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import grad

Let's write a function to replace all the different expressions and perform the taylor expansions 

In [4]:
def slow_rot(arg):
    subs_funcs = [(U, A), (V, B), (X, C), (Y, D)]
    
    if hasattr(arg, 'expr'):
        arg = arg.expr()
        
    if hasattr(arg, 'apply_map')*hasattr(arg, 'display'):
        for old_func, new_func in subs_funcs:
            arg.apply_map(lambda f: f.substitute_function(old_func, new_func))
        arg.apply_map(lambda f: f.taylor(a, 0, 1))
    else:
        for old_func, new_func in subs_funcs:
            arg = arg.substitute_function(old_func, new_func)
        arg = arg.taylor(a, 0, 1) 
        
    return arg

# I. Definition of the metric in ER

In [5]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


In [6]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

In [7]:
r_m, r_p, a, Q = var('r_m r_p a Q')

In [8]:
g = M.metric()
U = function('U')
V = function('V')
X = function('X')
Y = function('Y')
g[0,0] = U(r)
g[1,1] = V(r)
g[2,2] = X(r)
g[3,3] = Y(r,th) 

In [9]:
Psi(r) = (1-r_p/r)
Phi(r) = (1-r_m/r)
A(r) = -Psi(r)*Phi(r)**(7/13)
B(r) = 1/(Psi(r)*Phi(r)**(15/13))
C(r) = r**2*Phi(r)**(-2/13)
D(r) = (r*sin(th))**2*Phi(r)**(-2/13)

In [10]:
h = g.copy()
h.set_name('h')
h.apply_map(lambda f: f.substitute_function(U, A).substitute_function(V, B).substitute_function(X, C).substitute_function(Y, D)) 
h.display()

h = (-r_m/r + 1)^(7/13)*(r_p/r - 1) dt⊗dt - 1/((-r_m/r + 1)^(15/13)*(r_p/r - 1)) dr⊗dr + r^2/(-r_m/r + 1)^(2/13) dth⊗dth + r^2*sin(th)^2/(-r_m/r + 1)^(2/13) dph⊗dph

# II. The vector potential

Let's define the covariant derivative

In [11]:
nab = g.connection() 

and the potential vector field corresponding to a non rotating charged BH

In [12]:
pot_vec = M.tensor_field(0,1,name='A')
pot_vec[0]=0
pot_vec[1]=0
pot_vec[2]=0
pot_vec[3]=-sqrt(12*r_p*r_m/13) * cos(th)
pot_vec.display()

A = -2*sqrt(3/13)*sqrt(r_m*r_p)*cos(th) dph

## II.1. Definition of the EM tensor

We now derive the EM tensor:

$F_{\mu\nu}=\partial_{\mu}A_{\nu}-\partial_{\nu}A_{\mu}$

It can be obtained directly as the exterior derivative of the potential vector $A$ via the function `diff`
(or the method `exterior_derivative`)

In [13]:
F = diff(pot_vec)
F.set_name('F')
Fuu = F.up(g)
F.display()

F = 2/13*sqrt(13)*sqrt(3)*sqrt(r_m)*sqrt(r_p)*sin(th) dth∧dph

In the case of a magnetically charged BH, we must make the following transformation for the EM tensor:
$F_{\mu\nu}\rightarrow F'_{\mu\nu}=e^{-2\alpha\phi}\frac{1}{2}\epsilon^{\lambda\rho}_{\mu\nu}F_{\lambda\rho}^{el}$ and $\phi\rightarrow\phi'=-\phi$.

We first define the Levy-Civita tensor:

In [14]:
eps = g.volume_form()

## II.2. The scalar field

The scalar field is now expressed as $\phi'=-\phi$ where $\phi$ corresponds to the electrical case. 

In [15]:
Phi_sc = M.scalar_field((1-r_m/r)**(2/13), name='Phi_sc', latex_name=r'\Phi')
Phi_sc.display()

Phi_sc: M → ℝ
   (t, r, th, ph) ↦ (-r_m/r + 1)^(2/13)

The matter Lagrangian reads

In [16]:
Lm = -F['_ij']*F.up(g)['^ij']/2
Lm = Lm.expr()
Lm = Lm.taylor(a,0,1)

In [17]:
show(LatexExpr(r'\mathcal{L}_{m} = '), Lm, LatexExpr(r'+ \mathcal{O}(a^{2})'))

\mathcal{L}_{m} =  -12/13*r_m*r_p*sin(th)^2/(X(r)*Y(r, th)) + \mathcal{O}(a^{2})

# III. The motion equation  equation (1) 

## III.1. The left hand side of the motion equation : the Einstein tensor

In [18]:
ER_ricc = g.ricci()
ER_ricc.apply_map(lambda f: f.taylor(a,0,1)) 

In [19]:
ER_rsc = g.ricci_scalar()
ER_rsct = ER_rsc.expr().taylor(a,0,1)
show(LatexExpr(r'R = '), slow_rot(ER_rsc).factor(), LatexExpr(r'+ \mathcal{O}(a^{2})'))

R =  12/13*r_m*r_p*((r - r_m)/r)^(2/13)/r^4 + \mathcal{O}(a^{2})

We deduce the Einstein tensor

In [20]:
G = ER_ricc - 1/2*g*ER_rsc
G.apply_map(lambda f: f.taylor(a,0,1))

In [21]:
show(LatexExpr(r'G_{00} = '), slow_rot(G[0,0]).factor())

G_{00} =  -14/169*(r*r_m - 13*r*r_p + 12*r_m*r_p)*(r - r_p)*r_m/(r^6*((r - r_m)/r)^(4/13))

## III.2. The right hand side of the motion equation (1) 

### III.2.1 The first component: the stress-energy tensor $T_{\mu\nu}=2\left(F_{\rho\mu}F^{\rho}_{\hspace{0.2cm}\nu}-\frac{1}{4}g_{\mu\nu}F^{2}\right)$ 

In [22]:
Fuu = F.up(g)
F2 = F['_ab']*Fuu['^ab'] ; F2.display()

M → ℝ
(t, r, th, ph) ↦ 24/13*r_m*r_p*sin(th)^2/(X(r)*Y(r, th))

In [23]:
Fud = F.up(g,0)
T = 2*(F['_k.']*Fud['^k_.'] - 1/4*F2 * g)

Let's now define the stress-energy tensor

In [24]:
T.apply_map(lambda f: f.taylor(a,0,1))
show(LatexExpr(r'T_{\mu\nu} = '), slow_rot(T)[:])

T_{\mu\nu} =  [12/13*(r*r_m*r_p*(-r_m/r + 1)^(11/13) - r_m*r_p^2*(-r_m/r + 1)^(11/13))/r^5                                                                           0                                                                           0                                                                           0]
[                                                                          0    -12/13*r_m*r_p*(-r_m/r + 1)^(2/13)/(r^4 - r^3*r_m - (r^3 - r^2*r_m)*r_p)                                                                           0                                                                           0]
[                                                                          0                                                                           0                                       12/13*r_m*r_p*(-r_m/r + 1)^(2/13)/r^2                                                                           0]
[                                                                          0                                                                           0                                                                           0                             12/13*r_m*r_p*(-r_m/r + 1)^(2/13)*sin(th)^2/r^2]

We deduce the final quotient and perform a first-order Taylor expansion to simplify the expression before any further calculations

In [25]:
UU=T/Phi_sc
UU.apply_map(lambda f: f.taylor(a,0,1)) 

### III.2.3. The second component of the motion equation $\frac{1}{\Phi}(\triangledown_{\mu}\triangledown_{\nu}-g_{\mu\nu}\square)\Phi$

We now construct the d'Alembert operator

In [26]:
Dal = dalembertian(Phi_sc**2); Dal

Scalar field Box(Phi_sc^2) on the 4-dimensional Lorentzian manifold M

In [27]:
Gp = nab(Phi_sc**2); Gp 

1-form dPhi_sc^2 on the 4-dimensional Lorentzian manifold M

In [28]:
S = Phi_sc**(-2)*(nab(Gp).down(g)-g*Dal) 
S.apply_map(lambda f: f.taylor(a,0,1)) 

In [29]:
RHS = UU + S
RHS.apply_map(lambda f: f.taylor(a,0,1)) 

# IV. Verification of the field equations

## IV.1 Verification of the $G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi$ equation

Let's check that the two sides of the equation (1) are equal

In [30]:
slow_rot(G)==slow_rot(RHS)

True

## IV.2. Verification of the Maxewell equations $\triangledown\left(\sqrt{\phi}F_{\mu\nu}\right)$ 

In [31]:
Fuu.apply_map(lambda f: f.taylor(a,0,1))

In [32]:
q = Fuu*Phi_sc
q.apply_map(lambda f: f.taylor(a,0,1))

In [33]:
eq = nab(q)['_i^ij']
slow_rot(eq)

Vector field on the 4-dimensional Lorentzian manifold M

In [34]:
latex_str = r'\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{el} = ' 
show(LatexExpr(latex_str), eq[:])

\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{el} =  [0, 0, 0, 0]

## IV.3.  Verification of the $\phi = \frac{Lm}{R}$ equation

In [35]:
Phi_sc + slow_rot(Lm / ER_rsct) == 0

True

# In conclusion the field equations $G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi$ and  $\triangledown\left(\sqrt{\phi}T_{\mu\nu}\right)$ are perfectly verified at the first order.

# The magnetically charged solutions are solutions in ER as well for a non rotating black hole.